In [2]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn|
%pip install datasets
%pip install transformers
%pip install optuna

You should consider upgrading via the '/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "transformers"
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
zsh:

In [21]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset, load_metric
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,get_scheduler
from torch.optim import AdamW

from spacy import tokenizer
from spacy.lang.en import English
import torch
from torch.utils.data import DataLoader
from utils import RoBERT_Model,ToBERT_Model
import json

In [4]:
accuracy = load_metric('accuracy'); f1_score = load_metric('f1'); perplexity = load_metric('perplexity')

In [5]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Reusing dataset hyperpartisan_news_detection (/Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)
100%|██████████| 2/2 [00:00<00:00, 21.19it/s]


dict_keys(['train', 'validation'])

In [20]:
distribution_name = "hyperpartisan_distribution"
with open(distribution_name, 'r') as ins:
    distribution= json.load(ins)

seed = distribution['seed']

train_index = distribution['train']
eval_index = distribution['eval']
test_index = distribution['test']

In [19]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = seed)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = seed)

Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-66d6117f38d3c868.arrow
Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-31fcbf3d65e8fea2.arrow


With huggingface

In [7]:
eval_test_split = hyperpartisan_valid.train_test_split(test_size = 0.5)

hyperpartisan_eval = eval_test_split['train']
hyperpartisan_test = eval_test_split['test']

In [8]:
int_labels_train = {i: 1 if x == True else 0 for i,x in enumerate(hyperpartisan_train['hyperpartisan'])}
int_labels_eval = {i: 1 if x == True else 0 for i,x in enumerate(hyperpartisan_eval['hyperpartisan'])}
int_labels_test = {i: 1 if x == True else 0 for i,x in enumerate(hyperpartisan_test['hyperpartisan'])}

DOCUMENTS LESS THAN 300

In [23]:
#Documents that are <300 are categorised as short
#Documents that are >300 are categorised as BERT
#Documents that are >512 but <4096 are categorised long 

hyperpartisan_train

dict_keys(['Too Long', 'Long', 'BERT', 'Short'])

In [ ]:
#Should work once then modularize

max_input_length = 300 #Vary after obtaining end to end result
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #to pad sequences


def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=max_input_length)

hyperpartisan_train.select(train_index['short'])

hyperpartisan_train = hyperpartisan_train.map(tokenize)
hyperpartisan_valid = hyperpartisan_valid.map(tokenize)

In [ ]:
new_model = RoBERT_Model(,num_classes=2)

NO FINETUNING

In [ ]:
#with torch.no_grad():
#Obtain results of methods without finetuning
#obtain best parameter for this models.

EFFECT OF FINETUNING (with varying number of training files)

In [ ]:
def hp_space(trial):

  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.0001,0.0005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)
  }

In [ ]:
#Problems with finetuning with limited number of training files, is training set is unbalanced
#Less knowledge for neural network to learn from 
#Why not investigate the definition of what a chunk is, and use different pretraining objective e.g BERT,Roberta
#Problem with finetuning with long document is document input are limited in length, therefore learning across text documents is unbalanced

#Only way around finetuning on text document is to carefully analyse the training set and long documents.

In [ ]:
#for multi-class classification
#labels = hyperpartisan_train["label"]
#num_labels = len(labels)
#labels2id, id2labels = dict(),dict()
#for i,label in enumerate(labels):
    #labels2id[label] = i
    #id2labels[i] = label

results = {}
a_dataset = hyperpartisan_train.select(range(2000))
b_dataset = hyperpartisan_valid

#BERT models setup
max_input_length = 300 #Vary after obtaining end to end result
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def bert_init():
  return (BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length))

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=16, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)

#fine-tuning model

trainer = Trainer(model_init = bert_init, 
              model = model,
              args = training_args,
              train_dataset= a_dataset,
              eval_dataset= b_dataset,
              tokenizer= tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
best_run = trainer.hyperparameter_search(n_trials = 10, direction = "maximize", hp_space = hp_space)

#update hyperparameters 
for k,v in best_run.hyperparameters.items():
  setattr(training_args, k, v)

best_trainer = Trainer(model = model,
                       args = training_args,
                       train_dataset = a_dataset,
                       eval_dataset = b_dataset,
                       compute_metrics = compute_metrics,
                       data_collator = data_collator)

best_trainer.train()

AttributeError: ignored

In [ ]:
best_trainer.evaluate()

In [ ]:
#display result 
#display confusion matrix


Value(dtype='int64', id=None)

With pytorch 

In [ ]:
def select_files_to_dataloader(num,):
    
    """Num is the number of files to be finetuned"""
    
    small_train_dataset = hyperpartisan_train.shuffle(seed = 31).select(range(num))
    small_eval_dataset = hyperpartisan_valid.shuffle(seed = 31).select(range(num))

    train_dataloader = DataLoader(small_train_dataset, shuffle = True, batch_size = 8)
    eval_dataloader = DataLoader(small_eval_dataset, shuffle = True, batch_size= 8)

    return train_dataloader, eval_dataloader

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [ ]:
train_dataloader, eval_dataloader = select_files_to_dataloader(100)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)
optimizer = AdamW(model.parameters(), lr = 5e-5)

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps= 0, num_training_steps= num_training_steps)

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss()
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad() #prevents accumulation of gradients, probably useful in RNNs

        progress_bar.update(1)

LOADING AND PREPROCESSING

20 Newsgroup

In [ ]:
newsgroup_path = "20news-bydate"
element = os.listdir(newsgroup_path)

newsgroup_train_path = newsgroup_path + '/' + element[1]
newsgroup_test_path = newsgroup_path + '/' + element[0]

newsgroup_train = prepare_corpus(newsgroup_train_path)
newsgroup_test = prepare_corpus(newsgroup_test_path)

X_train_newsgroup_text = preproc(newsgroup_train)
X_test_newsgroup_text = preproc(newsgroup_test)

FileNotFoundError: ignored

In [ ]:
X_train_newsgroup_text.keys()

dict_keys(['talk.politics.mideast', 'rec.autos', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 'comp.os.ms-windows.misc', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'talk.politics.misc', 'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'talk.politics.guns', 'sci.space', 'soc.religion.christian', 'misc.forsale', 'talk.religion.misc'])